In [1]:
% reset -f

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
# np.set_printoptions(threshold='nan')
import pandas as pd

import glob

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from IPython.display import display

from collections import Counter

In [2]:
X = []
y = []

for j in range(10):
    fileNames = glob.glob('trainingDigits/' + str(j) + '_*.txt')
    for fN in fileNames:
        with open(fN, 'r') as f:
            rd = f.read()
            if '\r\n' in rd:
                rd = rd.replace('\r\n','')
            elif '\n' in rd:
                rd = rd.replace('\n','')
            X.append([int(i) for i in rd])
            y.append(j)

X = np.array(X)
y = np.array(y)

idx = range(X.shape[0])
np.random.shuffle(idx)

X = X[idx,:]
y = y[idx]

In [3]:
Xt = []
yt = []

for j in range(10):
    fileNames = glob.glob('testDigits/' + str(j) + '_*.txt')
    for fN in fileNames:
        with open(fN, 'r') as f:
            rd = f.read()
            if '\r\n' in rd:
                rd = rd.replace('\r\n','')
            elif '\n' in rd:
                rd = rd.replace('\n','')
            Xt.append([int(i) for i in rd])
            yt.append(j)

Xt = np.array(Xt)
yt = np.array(yt)

idx = range(Xt.shape[0])
np.random.shuffle(idx)

Xt = Xt[idx,:]
yt = yt[idx]

In [4]:
### GRID SEARCH


# # Create hidden layer size space
# hList = range(5,51,5)

# # Create alpha space (i.e. L2 regularization penalty parameter space)
# aList = []
# for k in [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]:
#     for j in [1,1.25,2.5,3.75,5,6.25,7.5,8.75]:
#         aList.append(j*k)

# # instantiate an MLP model to run the search on
# mlpModel = MLPClassifier(max_iter=200)

# # Create hyperparameter options
# hyperparameters = dict(hidden_layer_sizes=hList, alpha=aList)

# # run a grid search for hyperparameter tuning
# searchModel = GridSearchCV(mlpModel, hyperparameters, cv=3, verbose=1)
# best_model = searchModel.fit(X, y)

# # just out of curiosity, what was the result of the hyperparameter tuning
# bestHid = best_model.best_estimator_.get_params()['hidden_layer_sizes']
# bestReg = best_model.best_estimator_.get_params()['alpha']
# print 'Best hidden layer size:', bestHid
# print 'Best regularization parameter:', bestReg

In [5]:
### RANDOM SEARCH


# Create hidden layer size space
hSpace = range(5,201,5)
hList = []
for h1 in hSpace:
    for h2 in hSpace:
        for h3 in hSpace:
            hList.append((h1,h2,h3))


# Create alpha space (i.e. L2 regularization penalty parameter space)
aList = []
for k in [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]:
    for j in np.linspace(1,10,91):
        aList.append(j*k)

# instantiate an MLP model to run the search on
mlpModel = MLPClassifier(max_iter=200)

# Create hyperparameter options
hyperparameters = dict(hidden_layer_sizes=hList, alpha=aList)

rnd = np.random.randint(100)
searchModel = RandomizedSearchCV(mlpModel, hyperparameters, random_state=rnd, n_iter=60, cv=3, verbose=1, n_jobs=-1)
best_model = searchModel.fit(X, y)

# just out of curiosity, what was the result of the hyperparameter tuning
bestHid = best_model.best_estimator_.get_params()['hidden_layer_sizes']
bestReg = best_model.best_estimator_.get_params()['alpha']
print 'Best hidden layer size:', bestHid
print 'Best regularization parameter:', bestReg

Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  9.5min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed: 30.9min finished


Best hidden layer size: (155, 165, 185)
Best regularization parameter: 0.0039000000000000003


In [6]:
# prediction using the model from the search

# yp = best_model.predict(Xt)
yp = best_model.best_estimator_.predict(Xt)

100.0*sum(yp == yt)/yt.shape[0]

97.88583509513742

In [7]:
# prediction using a new model initialized with results from the search (for comparison)

new_mlpModel = MLPClassifier(hidden_layer_sizes=bestHid, alpha=bestReg, max_iter=200)
new_mlpModel.fit(X,y);

yp = new_mlpModel.predict(Xt)

100.0*sum(yp == yt)/yt.shape[0]

97.67441860465117